In [63]:
import pandas as pd
import matplotlib.pyplot as plt

In [64]:
crimes = pd.read_csv('~/Downloads/cleaned_chicago_crime_data.csv')

In [76]:
# get list of unique districts 1-25
districts = crimes['District'].unique().tolist()
districts

[11.0,
 7.0,
 2.0,
 9.0,
 5.0,
 6.0,
 4.0,
 12.0,
 15.0,
 14.0,
 1.0,
 20.0,
 10.0,
 22.0,
 8.0,
 17.0,
 3.0,
 18.0,
 16.0,
 25.0,
 24.0,
 19.0,
 21.0]

In [66]:
crime_types = crimes['Primary Type'].unique().tolist()
crime_types

['BATTERY',
 'OTHER OFFENSE',
 'ROBBERY',
 'NARCOTICS',
 'CRIMINAL DAMAGE',
 'WEAPONS VIOLATION',
 'THEFT',
 'BURGLARY',
 'MOTOR VEHICLE THEFT',
 'PUBLIC PEACE VIOLATION',
 'DECEPTIVE PRACTICE',
 'ASSAULT',
 'CRIMINAL TRESPASS',
 'CRIM SEXUAL ASSAULT',
 'INTERFERENCE WITH PUBLIC OFFICER',
 'ARSON',
 'LIQUOR LAW VIOLATION',
 'KIDNAPPING',
 'SEX OFFENSE',
 'OFFENSE INVOLVING CHILDREN',
 'PROSTITUTION',
 'HOMICIDE',
 'GAMBLING',
 'INTIMIDATION',
 'STALKING',
 'OBSCENITY',
 'PUBLIC INDECENCY',
 'HUMAN TRAFFICKING',
 'CONCEALED CARRY LICENSE VIOLATION',
 'OTHER NARCOTIC VIOLATION',
 'NON - CRIMINAL',
 'NON-CRIMINAL',
 'RITUALISM',
 'NON-CRIMINAL (SUBJECT SPECIFIED)',
 'DOMESTIC VIOLENCE']

In [67]:
violent_crimes = ['BATTERY','ASSAULT','CRIM SEXUAL ASSAULT', 'ARSON','HOMICIDE']
felonies = ['HUMAN TRAFFICKING', 'HOMICIDE','ARSON','KIDNAPPING','BURGLARY','ROBBERY','BATTERY','STALKING','SEX OFFENSE']

In [68]:
violents = []
for crime in violent_crimes:
    violents.append( crimes.loc[ crimes['Primary Type'] == crime] )

In [69]:
df_violent = violents[0]
for i in range(1,len(violents)):
    df_violent.append(violents[i])

In [70]:
df_violent;

In [71]:
The goal of this notebook is to analyze the hypothesis: does the location description, time of day, or district predict whether a crime will be violent or not

In [72]:
import pandas as pd

In [88]:
crimes = pd.read_csv('~/Downloads/cleaned_chicago_crime_data.csv')

In [89]:
crimes.head()

In [96]:
crimes.index = crimes['ID'] # make the index the actual case number
# remove columns that are unnecessary for the analysis and would probably just slow our code down
crimes = crimes.drop(['Beat', 'FBI Code','Updated On', 'Case Number', 'Block', 'IUCR', 'ID', 'Unnamed: 0','Description', 'Ward', 'X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude'], axis=1)

In [1]:
crime_types = crimes['Primary Type'].unique().tolist()

NameError: name 'crimes' is not defined

### Begin cleaning data

1. Remove NaN values
2. standardize the crime type to get rid of the uppercase, standardize the non-criminal type
3. standardize the location description

There are actually no Nan values to remove! Government data is reliable I guess. The only NaN values are location, but we still know the district, so as long as we don't end up needing location later on, this is fine

In [113]:
to_drop = crimes[crimes.isnull().any(axis=1)]
to_drop;

In [118]:
def standardize_primary_type(string):
    # compile all non-criminal offenses into on label
    if string == 'NON-CRIMINAL (SUBJECT SPECIFIED)' or string == 'NON - CRIMINAL' or string == 'NON-CRIMINAL':
        return 'non-criminal'
    if string == 'OTHER OFFENSE':
        return 'other'
    # rename crim sexual assault to just sexual assault to make it easier to read
    if string == 'CRIM SEXUAL ASSAULT':
        return 'sexual assault'
    else:
        # everything else, make sure to lowercase it so we don't have to use caps lock lol
        return string.lower()
        

In [82]:
def standardize_location(string):
    if type(string) == str:
        string = string.lower()
        if 'airport' in string:
            return 'airport'
    return string

In [ ]:
crimes['Primary Type'] = crimes['Primary Type'].apply(standardize_primary_type) # clean data 
crimes['location'] = crimes['Location Description'].apply(standardize_location)

In [17]:
# be careful if you run this one, only run it once it won't work the second time 
crimes = crimes.drop(['Location Description','Community Area'],axis=1)

In [104]:
crimes = crimes.rename(columns = {'Location': 'Coordinates'})

In [23]:
violents = ['BATTERY','ASSAULT','CRIM SEXUAL ASSAULT', 'ARSON','HOMICIDE']

In [108]:
violent_crimes = crimes[ (crimes['Primary Type'] == 'battery') | (crimes['Primary Type'] == 'assault') | (crimes['Primary Type'] == 'sexual assault') | (crimes['Primary Type'] == 'arson') | (crimes['Primary Type'] == 'homicide')]